In [4]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
# fixed
from animalsScript import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name
# fixed
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
# BRANDING REQUIREMENT #1: Logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div([
        #FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        # REQUIREMENT: Dashboard widget dropdown featuring:
        # water rescue, mountain/wilderness rescue, disaster rescue, and reset button
        dcc.RadioItems(
            id='radioButtons',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain/Wilderness', 'value': 'mountain'},
                {'label': 'Disaster', 'value':'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ], 
            value='reset'
        )]
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
# fixed
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
    # BRANDING REQUIREMENTS #2: Developer name/identifier
    html.Center(html.B('Design by Ken Rodriguez')),

])

#############################################
# Interaction Between Components / Controller
#############################################



    
#@app.callback([Output('datatable-id','data'),
#               Output('datatable-id','columns')],
#              [Input('filter-type', 'value')])
#def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#        
#        
#        return (data,columns)

#
# Radio Button Sort
#
@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('radioButtons', 'value')])
def update_output(value):
    # Preferred WATER dogs: Labrador, Chesapeake, Newfoundland. Female. 26 to 156 wks.
    if (value=='water'):
        df = pd.DataFrame(list(shelter.readAll({
            "breed": {
                "$in":[
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks":{
                        "$lte": 156.00
                    }
                }
            ]
            }
        )))
    # Preferred MOUNTAIN dogs: German Shepherd, Malamute, Old English Sheepdog, Husky, Rottweiler
    # intact male, 26 to 156 wks.
    elif (value == 'mountain'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    # Preferred DISASTER dogs: pinscher, german shepherd, golden retriever, bloodhound, rottweiler
    # intact male, 20 to 300 wks.
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.readAll(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.readAll({}))
    return df.to_dict('records')

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_viewport_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    return[
        dcc.Graph(
        figure = px.pie(
        data_frame=dff,
        values = values,
        names = names,
        color_discrete_sequence=px.colors.sequential.RdBu,
        width = 800,
        height = 500))
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.4,-97.3], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # set position to lat/long from database entry in the first row
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[1,4]),
                dl.Popup([
                    html.H1(dff.iloc[1,9]),
                    html.P()
                ])
            ]) 
        ])
    ]


app